In [1]:
import cv2, os
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from sklearn.model_selection import train_test_split
from keras.utils import plot_model
from model.u_net import get_unet_128, get_unet_256, get_unet_512, get_unet_1024
from keras.optimizers import RMSprop
from model.losses import bce_dice_loss, dice_loss, weighted_bce_dice_loss, weighted_dice_loss, dice_coeff
from keras.utils import multi_gpu_model
from keras.models import Model

/Users/lochappy/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/lochappy/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/lochappy/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/lochappy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/lochappy/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: Runt

# Configuration

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

input_height = 128
input_width = 128

m = get_unet_128()

max_epochs = 100
batch_size = 16
threshold = 0.5

m.summary()
plot_model( m , show_shapes=True , to_file='unet128.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 128, 128, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

# Data input

In [ ]:
AIRBUS_SHIP_TRAIN_DIR = "../Data/train"
AIRBUS_SHIP_MASK_TRAIN_DIR = "../Data/train_mask"

AIRBUS_SHIP_VAL_DIR = "../Data/val"
AIRBUS_SHIP_MASK_VAL_DIR = "../Data/val_mask"

In [ ]:
def getImageMaskPair(dataFoler, maskFoler):
    """ return the list pair [[data,mask], [data,mask]...]"""
    data = sorted(glob.glob(dataFoler + "/*.jpg") + glob.glob(dataFoler + "/*.png"))
    mask = sorted(glob.glob(maskFoler + "/*.jpg") + glob.glob(maskFoler + "/*.png"))
    return [[imgPath,mask] for imgPath, mask in zip(data,mask) 
                                if imgPath.split('/')[-1].split('.')[0] == mask.split('/')[-1].split('.')[0] ]

TRAIN_DATA = getImageMaskPair(AIRBUS_SHIP_TRAIN_DIR,AIRBUS_SHIP_MASK_TRAIN_DIR)
print("There are {} training images found.".format(len(TRAIN_DATA)))

VAL_DATA = getImageMaskPair(AIRBUS_SHIP_VAL_DIR,AIRBUS_SHIP_MASK_VAL_DIR)
print("There are {} validating images found.".format(len(VAL_DATA)))

In [ ]:
def randomHueSaturationValue(image, hue_shift_limit=(-180, 180),
                             sat_shift_limit=(-255, 255),
                             val_shift_limit=(-255, 255), u=0.5):
    if np.random.random() < u:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(image)
        hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
        h = cv2.add(h, hue_shift)
        sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
        s = cv2.add(s, sat_shift)
        val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
        v = cv2.add(v, val_shift)
        image = cv2.merge((h, s, v))
        image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)

    return image


def randomShiftScaleRotate(image, mask,
                           shift_limit=(-0.0625, 0.0625),
                           scale_limit=(-0.1, 0.1),
                           rotate_limit=(-45, 45), aspect_limit=(0, 0),
                           borderMode=cv2.BORDER_CONSTANT, u=0.5):
    if np.random.random() < u:
        height, width, channel = image.shape

        angle = np.random.uniform(rotate_limit[0], rotate_limit[1])  # degree
        scale = np.random.uniform(1 + scale_limit[0], 1 + scale_limit[1])
        aspect = np.random.uniform(1 + aspect_limit[0], 1 + aspect_limit[1])
        sx = scale * aspect / (aspect ** 0.5)
        sy = scale / (aspect ** 0.5)
        dx = round(np.random.uniform(shift_limit[0], shift_limit[1]) * width)
        dy = round(np.random.uniform(shift_limit[0], shift_limit[1]) * height)

        cc = np.math.cos(angle / 180 * np.math.pi) * sx
        ss = np.math.sin(angle / 180 * np.math.pi) * sy
        rotate_matrix = np.array([[cc, -ss], [ss, cc]])

        box0 = np.array([[0, 0], [width, 0], [width, height], [0, height], ])
        box1 = box0 - np.array([width / 2, height / 2])
        box1 = np.dot(box1, rotate_matrix.T) + np.array([width / 2 + dx, height / 2 + dy])

        box0 = box0.astype(np.float32)
        box1 = box1.astype(np.float32)
        mat = cv2.getPerspectiveTransform(box0, box1)
        image = cv2.warpPerspective(image, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                    borderValue=(
                                        0, 0,
                                        0,))
        mask = cv2.warpPerspective(mask, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                   borderValue=(
                                       0, 0,
                                       0,))

    return image, mask


def randomHorizontalFlip(image, mask, u=0.5):
    if np.random.random() < u:
        image = cv2.flip(image, 1)
        mask = cv2.flip(mask, 1)

    return image, mask

class TrainDataGenerator:
    def __init__(self, listOfImagePaths, batch_size, target_size):
        self.dataGen = self._train_generator(listOfImagePaths, batch_size, target_size)
        self.len = np.ceil(float(len(listOfImagePaths)) / float(batch_size))
        
    def __len__(self):
        return self.len
    
    def __next__(self):
        return self.dataGen.next()
    
    def _train_generator(self, dataList, batch_size, input_size):
        while True:
            for start in range(0, len(dataList), batch_size):
                x_batch = []
                y_batch = []
                end = min(start + batch_size, len(dataList))
                train_batch = dataList[start:end]
                for datum in train_batch:
                    img = cv2.imread(datum[0])
                    img = cv2.resize(img, (input_size, input_size))
                    mask = cv2.imread(datum[1], cv2.IMREAD_GRAYSCALE)*255
                    mask = cv2.resize(mask, (input_size, input_size))
                    img = randomHueSaturationValue(img,
                                                   hue_shift_limit=(-50, 50),
                                                   sat_shift_limit=(-5, 5),
                                                   val_shift_limit=(-15, 15))
                    img, mask = randomShiftScaleRotate(img, mask,
                                                       shift_limit=(-0.0625, 0.0625),
                                                       scale_limit=(-0.1, 0.1),
                                                       rotate_limit=(-0, 0))
                    img, mask = randomHorizontalFlip(img, mask)
                    mask = np.expand_dims(mask, axis=2)
                    x_batch.append(img)
                    y_batch.append(mask)
                x_batch = np.array(x_batch, np.float32) / 255
                y_batch = np.array(y_batch, np.float32) / 255
                yield x_batch, y_batch

                
class ValDataGenerator:
    def __init__(self, listOfImagePaths, batch_size, target_size):
        self.dataGen = self._valid_generator(listOfImagePaths, batch_size, target_size)
        self.len = np.ceil(float(len(listOfImagePaths)) / float(batch_size))
        
    def __len__(self):
        return self.len
    
    def __next__(self):
        return self.dataGen.next()
    
    def _valid_generator(self, dataList, batch_size, input_size):
        while True:
            for start in range(0, len(dataList), batch_size):
                x_batch = []
                y_batch = []
                end = min(start + batch_size, len(dataList))
                val_batch = dataList[start:end]
                for datum in val_batch:
                    img = cv2.imread(datum[0])
                    img = cv2.resize(img, (input_size, input_size))
                    mask = cv2.imread(datum[1], cv2.IMREAD_GRAYSCALE)*255
                    mask = cv2.resize(mask, (input_size, input_size))
                    mask = np.expand_dims(mask, axis=2)
                    x_batch.append(img)
                    y_batch.append(mask)
                x_batch = np.array(x_batch, np.float32) / 255
                y_batch = np.array(y_batch, np.float32) / 255
                yield x_batch, y_batch

# Train

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/best_weights.hdf5',
                             save_best_only=True,
                             save_weights_only=True),
             TensorBoard(log_dir='logs')]


In [ ]:
m.fit_generator(generator=train_generator(),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(batch_size)),
                    epochs=epochs,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(batch_size)

In [ ]:
num_gpus = len(os.environ["CUDA_VISIBLE_DEVICES"].split(","))
#paralell_m = multi_gpu_model(m, gpus=2)
if num_gpus >= 2:
    paralell_m = multi_gpu_model(m, gpus=num_gpus)
else:
    paralell_m = m
paralell_m.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])

In [ ]:
model_name = "0_unet_128"
save_weights_path = "weights/"

epochs = 30
if num_gpus >= 2:
    train_batch_size = 8
else:
    train_batch_size = 4
val_batch_size = 4


G_train = DataSequence(TRAIN_DATA,batch_size=train_batch_size,input_height=input_width,input_width=input_width)
train_steps = len(G_train)
G_val = DataSequence(VAL_DATA,batch_size=val_batch_size,input_height=input_height,input_width=input_width)
val_steps = len(G_val)

for ep in range(1,epochs):
    history = paralell_m.fit_generator( G_train, train_steps, \
                                        validation_data=G_val , \
                                        validation_steps=val_steps, \
                                        epochs=1, \
                                        max_queue_size=20, workers=8, \
                                        use_multiprocessing=False )
    weight_path = save_weights_path + model_name + \
                "_dropout_weights_ep{:02d}_train{:.4f}_val{:.4f}.h5".format(ep, \
                history.history['mean_squared_error'][0],history.history['val_mean_squared_error'][0])
    m.save_weights(weight_path)
    print("Saving model to {}".format(weight_path))